In [ ]:

import requests
import time

# ✅ Kakao REST API 키 입력 (직접 발급 필요)
KAKAO_API_KEY = "-"

# ✅ Kakao API
headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}

In [1]:
import pandas as pd

vol_df1= pd.read_csv("4_weekday.csv")
vol_df1

,spot_num,date,hour,inout,lane,volume
0,A-01,20250501,0,1,1,198
1,A-01,20250501,0,1,2,309
2,A-01,20250501,0,2,1,490
3,A-01,20250501,0,2,2,328
4,A-02,20250501,0,1,1,169
...,...,...,...,...,...,...
456898,F-09,20250530,23,2,2,239
456899,F-10,20250530,23,1,1,278
456900,F-10,20250530,23,1,2,236
456901,F-10,20250530,23,2,1,834


In [2]:
vol_df2= pd.read_csv("4_weekend.csv")
vol_df2

,spot_num,date,hour,inout,lane,volume
0,A-01,20250405,0,1,1,188
1,A-01,20250405,0,1,2,308
2,A-01,20250405,0,2,1,396
3,A-01,20250405,0,2,2,271
4,A-02,20250405,0,1,1,165
...,...,...,...,...,...,...
161022,F-09,20250427,23,2,2,121
161023,F-10,20250427,23,1,1,182
161024,F-10,20250427,23,1,2,173
161025,F-10,20250427,23,2,1,293


In [3]:
total_vol_df = pd.concat([vol_df1,vol_df2]).reset_index()
total_vol_df

,index,spot_num,date,hour,inout,lane,volume
0,0,A-01,20250501,0,1,1,198
1,1,A-01,20250501,0,1,2,309
2,2,A-01,20250501,0,2,1,490
3,3,A-01,20250501,0,2,2,328
4,4,A-02,20250501,0,1,1,169
...,...,...,...,...,...,...,...
617925,161022,F-09,20250427,23,2,2,121
617926,161023,F-10,20250427,23,1,1,182
617927,161024,F-10,20250427,23,1,2,173
617928,161025,F-10,20250427,23,2,1,293


In [4]:
spot_df= pd.read_csv("spot_with_address.csv")
spot_df

,spot_num,spot_name,x,y,lon,lat,구,동,지역
0,A-01,성산로(금화터널),195489.000000,452136.000000,126.946835,37.571502,서대문구,봉원동,북서부
1,A-02,사직로(사직터널),196756.776106,452546.638644,126.961184,37.575207,종로구,무악동,북서부
2,A-03,자하문로(자하문터널),197216.855046,454350.990432,126.966385,37.591465,종로구,청운동,북서부
3,A-04,대사관로(삼청터널),198648.893154,455200.108465,126.982598,37.599119,성북구,성북동,북서부
4,A-05,율곡로(안국역),198645.671347,452937.216603,126.982566,37.578731,종로구,안국동,북서부
...,...,...,...,...,...,...,...,...,...
134,F-06,경부고속도로,202107.000000,443264.000000,127.021724,37.491575,서초구,서초동,남동부
135,F-07,분당수서로,207716.000000,444241.000000,127.085158,37.500348,송파구,삼전동,남동부
136,F-08,강남순환로(관악터널),191832.000000,437667.000000,126.905601,37.441111,금천구,시흥동,남서부
137,F-09,서부간선지하도로,189706.000000,441044.000000,126.881529,37.471517,금천구,가산동,남서부


In [5]:
import pandas as pd

# ✅ 병합
merged_df = pd.merge(total_vol_df, spot_df, on="spot_num", how="left")

# ✅ 필요한 열 선택 및 결측치 제거 (선제적 정리)
# merged_df = merged_df[["spot_num", "spot_name", "date", "hour", "inout", "volume", "구", "동", "지역"]]
# merged_df.dropna(subset=["지역", "volume"], inplace=True)

# ✅ 지역별 혼잡도 집계 (유입, 유출 따로)
region_gb = (
    merged_df.groupby(["date", "hour","구", "동", "지역",  "spot_name",'lon',	'lat'])[["volume"]]
    .sum()
    .reset_index()
    .rename(columns={"volume": "총교통량"})
)

# # ✅ 유입/유출 해석 이름 붙이기
# region_gb["구분"] = region_gb["inout"].map({1: "유입", 2: "유출"})

# ✅ 시간대별 정렬
region_gb = region_gb.sort_values(by=["date", "hour", "지역"])

# ✅ 예시 저장
# region_gb.to_csv("지역별_시간별_교통량.csv", index=False, encoding="utf-8-sig")
# print("✅ 저장 완료: 지역별_시간별_교통량.csv")


In [6]:
region_gb

,date,hour,구,동,지역,spot_name,lon,lat,총교통량
0,20250405,0,강남구,삼성동,남동부,테헤란로(선릉역),127.050051,37.508634,1955
1,20250405,0,강남구,세곡동,남동부,밤고개로(세곡동사거리),127.105634,37.465363,332
2,20250405,0,강남구,세곡동,남동부,헌릉로(세곡동사거리),127.103285,37.467617,463
3,20250405,0,강남구,역삼동,남동부,언주로(매봉터널),127.045378,37.495175,937
5,20250405,0,강동구,천호동,남동부,광진교,127.113094,37.546933,190
...,...,...,...,...,...,...,...,...,...
94704,20250530,23,중구,봉래동1가,북서부,소월로(회현역),126.974277,37.559852,146
94705,20250530,23,중구,순화동,북서부,세종대로(서울역),126.972176,37.561329,1864
94706,20250530,23,중구,순화동,북서부,칠패로(숭례문),126.970502,37.562547,688
94710,20250530,23,중구,정동,북서부,서소문로(시청역),126.970693,37.565451,1462


In [7]:

# ✅ 데이터 불러오기 및 전처리 (동일)
merged_df = region_gb.copy()
merged_df["date"] = pd.to_datetime(merged_df["date"], format="%Y%m%d")
merged_df["weekday"] = merged_df["date"].dt.day_name()

day_map = {
    "Monday": "월요일",
    "Tuesday": "화요일",
    "Wednesday": "수요일",
    "Thursday": "목요일",
    "Friday": "금요일",
    "Saturday": "토요일",
    "Sunday": "일요일"
}
merged_df["weekday"] = merged_df["weekday"].map(day_map)


In [8]:
# ✅ 출근시간대(평일 7~9시)
commute_hours = [7,8,9]
weekdays_df = merged_df[merged_df["weekday"].isin(["월요일", "화요일", "수요일", "목요일", "금요일"])]
commute_df = weekdays_df[weekdays_df["hour"].isin(commute_hours)]

commute_top10 = commute_df.groupby("spot_name")["총교통량"].median().reset_index()
commute_top10 = commute_top10.sort_values("총교통량", ascending=False).head(10)

# 중앙 좌표까지 merge
commute_top10 = commute_top10.merge(
    merged_df.groupby("spot_name")[["lon","lat"]].median().reset_index(),
    on="spot_name",
    how="left"
)

print("🚗 평일 출근시간대 top10")
display(commute_top10)

# ✅ 퇴근시간대(평일 16~18시)
leave_hours = [16,17,18]
leave_df = weekdays_df[weekdays_df["hour"].isin(leave_hours)]

leave_top10 = leave_df.groupby("spot_name")["총교통량"].median().reset_index()
leave_top10 = leave_top10.sort_values("총교통량", ascending=False).head(10)

leave_top10 = leave_top10.merge(
    merged_df.groupby("spot_name")[["lon","lat"]].median().reset_index(),
    on="spot_name",
    how="left"
)

print("🚗 평일 퇴근시간대 top10")
display(leave_top10)


# ✅ 주말(토/일)

weekend_hours = [10,11,12,13,14,15,16,17]
weekend_df = merged_df[merged_df["weekday"].isin(["토요일", "일요일"])]
weekend_df = weekend_df[weekend_df["hour"].isin(weekend_hours)]

weekend_top10 = weekend_df.groupby("spot_name")["총교통량"].median().reset_index()
weekend_top10 = weekend_top10.sort_values("총교통량", ascending=False).head(10)

weekend_top10 = weekend_top10.merge(
    merged_df.groupby("spot_name")[["lon","lat"]].median().reset_index(),
    on="spot_name",
    how="left"
)

print("🚗 주말 top10")
display(weekend_top10)


🚗 평일 출근시간대 top10


,spot_name,총교통량,lon,lat
0,강변북로(난지한강공원),13112.0,126.870423,37.573624
1,올림픽대로,13063.0,126.971274,37.508909
2,강변북로,11949.5,126.972209,37.519784
3,올림픽대로(강일IC),11583.0,127.155176,37.572572
4,한남대교,10340.0,127.010897,37.529688
5,경부고속도로(양재IC),9653.0,127.036491,37.467757
6,경부고속도로,9578.5,127.021724,37.491575
7,양재대로(양재IC),8758.0,127.032950,37.462920
8,경인고속국도(신월IC),8171.0,126.830250,37.527884
9,성산대교,7942.0,126.889084,37.554839


🚗 평일 퇴근시간대 top10


,spot_name,총교통량,lon,lat
0,강변북로(난지한강공원),13989.0,126.870423,37.573624
1,올림픽대로(강일IC),11975.0,127.155176,37.572572
2,올림픽대로,11266.0,126.971274,37.508909
3,올림픽대로(개화IC),9596.5,126.810894,37.590591
4,경부고속도로(양재IC),9535.0,127.036491,37.467757
5,경인고속국도(신월IC),9273.5,126.830250,37.527884
6,양재대로(양재IC),9178.5,127.032950,37.462920
7,강변북로,9075.0,126.972209,37.519784
8,경부고속도로,8503.0,127.021724,37.491575
9,한남대교,8489.0,127.010897,37.529688


🚗 주말 top10


,spot_name,총교통량,lon,lat
0,강변북로(난지한강공원),13265.5,126.870423,37.573624
1,올림픽대로(강일IC),11854.5,127.155176,37.572572
2,올림픽대로,11752.0,126.971274,37.508909
3,강변북로,11595.5,126.972209,37.519784
4,경인고속국도(신월IC),9058.5,126.830250,37.527884
5,경부고속도로,9005.0,127.021724,37.491575
6,경부고속도로(양재IC),8999.0,127.036491,37.467757
7,올림픽대로(개화IC),8993.0,126.810894,37.590591
8,한남대교,8702.0,127.010897,37.529688
9,강변북로(구리시계),8060.5,127.115161,37.566149


In [9]:
df = merged_df

# ✅ 괄호 제거해서 도로명만 추출
df["도로명"] = df["spot_name"].str.replace(r"\(.*?\)", "", regex=True).str.strip()

# ✅ 평일만 필터링
weekdays_df = df[df["weekday"].isin(["월요일", "화요일", "수요일", "목요일", "금요일"])]

# ✅ 출근시간대
commute_hours = [7,8,9]
commute_df = weekdays_df[weekdays_df["hour"].isin(commute_hours)]

# ✅ 퇴근시간대
leave_hours = [16,17,18]
leave_df = weekdays_df[weekdays_df["hour"].isin(leave_hours)]



# ▶ 중앙값 기준 Top10 (도로명 기준)
commute_df_median = commute_df.groupby("도로명")["총교통량"].median().reset_index()
top10_commute_df_median = commute_df_median.sort_values("총교통량", ascending=False).head(10)
top10_commute_df_median = top10_commute_df_median.reset_index(drop=True)
top10_commute_df_median.index += 1

print("🚗 평일_출근시간대 도로별 총교통량 Top10 (도로명 기준)")
display(top10_commute_df_median)

leave_df_median = leave_df.groupby("도로명")["총교통량"].median().reset_index()
top10_leave_df_median = leave_df_median.sort_values("총교통량", ascending=False).head(10)
top10_leave_df_median = top10_leave_df_median.reset_index(drop=True)
top10_leave_df_median.index += 1

print("🚗 평일_퇴근시간대 도로별 총교통량 Top10 (도로명 기준)")
display(top10_leave_df_median)


weekend_hours = [10,11,12,13,14,15,16,17]
weekend_df = df[df["weekday"].isin(["토요일", "일요일"])]
weekend_df = weekend_df[weekend_df["hour"].isin(weekend_hours)]

weekend_median = weekend_df.groupby("도로명")["총교통량"].median().reset_index()
top10_weekend_median = weekend_median.sort_values("총교통량", ascending=False).head(10)
top10_weekend_median = top10_weekend_median.reset_index(drop=True)
top10_weekend_median.index += 1

print("🚗 주말(토/일) 도로별 총교통량 Top10 (도로명 기준)")
display(top10_weekend_median)


🚗 평일_출근시간대 도로별 총교통량 Top10 (도로명 기준)


,도로명,총교통량
1,강변북로,11849.0
2,올림픽대로,11331.0
3,한남대교,10340.0
4,경부고속도로,9641.0
5,양재대로,8758.0
6,경인고속국도,8171.0
7,성산대교,7942.0
8,내부순환로,7255.0
9,분당수서로,6952.0
10,마포대교,6733.0


🚗 평일_퇴근시간대 도로별 총교통량 Top10 (도로명 기준)


,도로명,총교통량
1,올림픽대로,11138.0
2,경인고속국도,9273.5
3,양재대로,9178.5
4,강변북로,9078.0
5,경부고속도로,9002.5
6,한남대교,8489.0
7,내부순환로,6978.5
8,분당수서로,6958.0
9,행주대교,6843.5
10,북부간선도로,6724.5


🚗 주말(토/일) 도로별 총교통량 Top10 (도로명 기준)


,도로명,총교통량
1,올림픽대로,11462.0
2,강변북로,11408.0
3,경인고속국도,9058.5
4,경부고속도로,9001.5
5,한남대교,8702.0
6,양재대로,7936.0
7,내부순환로,6953.5
8,분당수서로,6697.0
9,성산대교,6684.5
10,북부간선도로,6489.5


In [11]:
import pandas as pd
import requests
import random
import numpy as np
import time
from math import radians, cos, sin, asin, sqrt

np.random.seed(20250744)
random.seed(20250744)

# ✅ Haversine 거리 (미터)
def haversine(lon1, lat1, lon2, lat2):
    R = 6371000
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    return R * c

# ✅ 서울 외곽 후보
def get_outer_start_candidates():
    mid_lon = merged_df["lon"].mean()
    mid_lat = merged_df["lat"].mean()
    range_lon = merged_df["lon"].max() - merged_df["lon"].min()
    range_lat = merged_df["lat"].max() - merged_df["lat"].min()
    buffer_lon = range_lon * 0.2
    buffer_lat = range_lat * 0.2
    outer = merged_df[
        (merged_df["lon"] > mid_lon + buffer_lon) | (merged_df["lon"] < mid_lon - buffer_lon) |
        (merged_df["lat"] > mid_lat + buffer_lat) | (merged_df["lat"] < mid_lat - buffer_lat)
    ]
    return outer.drop_duplicates(subset=["spot_name"])


def get_real_apartment_poi(lon, lat):
    brands = ["래미안","자이","힐스테이트","푸르지오","e편한세상","아이파크","롯데캐슬","더샵",
              "센트레빌","벽산블루밍","금호어울림","두산위브","한신휴플러스","꿈에그린","포레나",
              "휴먼시아","주공","리센츠","트리마제","파크리오","아크로리버","마포래미안푸르지오",
              "해링턴플레이스","쌍용예가","우미린","계룡리슈빌","한신더휴","SK뷰","아파트","빌라","주택"]
    
    for radius in [2000,3000,4000,5000]:
        for keyword in brands:
            try:
                res = requests.get(
                    "https://dapi.kakao.com/v2/local/search/keyword.json",
                    headers=headers,
                    params={"query": keyword, "x": lon, "y": lat, "radius": radius, "size": 15},
                    timeout=5
                )
                places = res.json().get("documents", [])
                # ✅ "예정" 포함 제외
                places = [p for p in places 
                          if any(k in p["place_name"] for k in ["아파트","빌라","주택"])
                          and "예정" not in p["place_name"]]
                if places:
                    return random.choice(places)
            except:
                time.sleep(1)
    return None


# ✅ 회사 POI (회사 겹치지 않게)
def get_company_office_poi_unique(lon, lat, exclude_ids, existing_address=None):
    must_keywords = ["타워", "본사", "빌딩"]
    exclude_words = ["아파트","빌라","주택","주민센터","부동산",'어린이집','R&D']
    radius_list = [1000,2000,3000,4000,5000,6000,7000,8000]

    for radius in radius_list:
        for keyword in must_keywords:
            try:
                res = requests.get(
                    "https://dapi.kakao.com/v2/local/search/keyword.json",
                    headers=headers,
                    params={"query": keyword, "x": lon, "y": lat, "radius": radius, "size":15},
                    timeout=5
                )
                places = res.json().get("documents", [])
                filtered = []
                for p in places:
                    pname = p["place_name"]
                    addr = p["road_address_name"] or p["address_name"]
                    if not any(w in pname for w in must_keywords):
                        continue
                    if any(w in pname for w in exclude_words):
                        continue
                    if existing_address and addr == existing_address:
                        continue
                    if p["id"] in exclude_ids:
                        continue
                    filtered.append(p)
                if filtered:
                    return random.choice(filtered)
            except:
                time.sleep(1)
    return None

# ✅ 출근 경로
def make_outer_to_center_routes(way_row, count=5, min_distance_m=3000):
    outer_spots = get_outer_start_candidates()
    results, used_pairs, used_work_ids, attempts = [], set(), set(), 0
    while len(results) < count and attempts < 10000:
        attempts += 1
        start_row = outer_spots.sample(1).iloc[0]
        poi_home = get_real_apartment_poi(start_row["lon"], start_row["lat"])
        poi_work = get_company_office_poi_unique(
            way_row["lon"], way_row["lat"], used_work_ids,
            existing_address=poi_home["road_address_name"] if poi_home else None
        )
        if not poi_home or not poi_work: continue
        distance = haversine(float(poi_home["x"]), float(poi_home["y"]), float(poi_work["x"]), float(poi_work["y"]))
        if distance < min_distance_m: continue
        pair_key = (poi_home["id"], poi_work["id"])
        if pair_key in used_pairs: continue
        used_work_ids.add(poi_work["id"])
        results.append({
            "type":"출근",
            "출발_건물명": poi_home["place_name"],
            "출발_주소": poi_home["road_address_name"],
            "출발_lon": float(poi_home["x"]),
            "출발_lat": float(poi_home["y"]),
            "경유_도로명": way_row["spot_name"],
            "경유_lon": way_row["lon"],
            "경유_lat": way_row["lat"],
            "도착_건물명": poi_work["place_name"],
            "도착_주소": poi_work["road_address_name"],
            "도착_lon": float(poi_work["x"]),
            "도착_lat": float(poi_work["y"])
        })
        used_pairs.add(pair_key)
    return pd.DataFrame(results)

# ✅ 퇴근 경로
def make_center_to_outer_routes(way_row, count=5, min_distance_m=3000):
    outer_spots = get_outer_start_candidates()
    results, used_pairs, used_work_ids, attempts = [], set(), set(), 0
    while len(results) < count and attempts < 10000:
        attempts += 1
        end_row = outer_spots.sample(1).iloc[0]
        poi_home = get_real_apartment_poi(end_row["lon"], end_row["lat"])
        poi_work = get_company_office_poi_unique(
            way_row["lon"], way_row["lat"], used_work_ids,
            existing_address=poi_home["road_address_name"] if poi_home else None
        )
        if not poi_home or not poi_work: continue
        distance = haversine(float(poi_home["x"]), float(poi_home["y"]), float(poi_work["x"]), float(poi_work["y"]))
        if distance < min_distance_m: continue
        pair_key = (poi_home["id"], poi_work["id"])
        if pair_key in used_pairs: continue
        used_work_ids.add(poi_work["id"])
        results.append({
            "type":"퇴근",
            "출발_건물명": poi_work["place_name"],
            "출발_주소": poi_work["road_address_name"],
            "출발_lon": float(poi_work["x"]),
            "출발_lat": float(poi_work["y"]),
            "경유_도로명": way_row["spot_name"],
            "경유_lon": way_row["lon"],
            "경유_lat": way_row["lat"],
            "도착_건물명": poi_home["place_name"],
            "도착_주소": poi_home["road_address_name"],
            "도착_lon": float(poi_home["x"]),
            "도착_lat": float(poi_home["y"])
        })
        used_pairs.add(pair_key)
    return pd.DataFrame(results)

# ✅ Top10 도출
commute_top10 = (merged_df[merged_df["hour"].isin([7,8,9])]
    .groupby("spot_name")["총교통량"].median()
    .reset_index()
    .sort_values("총교통량", ascending=False)
    .head(10)
    .merge(merged_df.groupby("spot_name")[["lon","lat"]].median().reset_index(), on="spot_name"))

leave_top10 = (merged_df[merged_df["hour"].isin([16,17,18])]
    .groupby("spot_name")["총교통량"].median()
    .reset_index()
    .sort_values("총교통량", ascending=False)
    .head(10)
    .merge(merged_df.groupby("spot_name")[["lon","lat"]].median().reset_index(), on="spot_name"))

# ✅ 출근 경로
commute_routes = pd.concat([
    make_outer_to_center_routes(pd.Series({"spot_name": r["spot_name"], "lon": r["lon"], "lat": r["lat"]}), count=5)
    for _, r in commute_top10.iterrows()
]).reset_index(drop=True)

# ✅ 퇴근 경로
leave_routes = pd.concat([
    make_center_to_outer_routes(pd.Series({"spot_name": r["spot_name"], "lon": r["lon"], "lat": r["lat"]}), count=5)
    for _, r in leave_top10.iterrows()
]).reset_index(drop=True)

commute_routes, leave_routes


(   type                     출발_건물명                 출발_주소      출발_lon  \
 0    출근                당산래미안아파트 남문                        126.902219   
 1    출근              문정래미안아파트 120동         서울 송파구 문정로 83  127.129843   
 2    출근             래미안파크스위트아파트 후문       서울 광진구 광나루로 545  127.089418   
 3    출근               동부센트레빌아파트 상가     서울 강서구 방화대로45길 32  126.814068   
 4    출근            강서힐스테이트아파트 게이트1                        126.837658   
 5    출근                남서울힐스테이트아파트       서울 금천구 시흥대로 165  126.901097   
 6    출근            래미안장안2차아파트 215동     서울 동대문구 장안벚꽃로 167  127.074949   
 7    출근                      래미안빌라     서울 양천구 목동중앙로3길 83  126.867255   
 8    출근          잠실래미안아이파크아파트 123동                        127.108773   
 9    출근               평창동롯데캐슬로잔아파트      서울 종로구 평창문화로 156  126.977399   
 10   출근  구리 아름마을인창래미안5단지아파트 전기차충전소    경기 구리시 동구릉로85번길 76  127.132534   
 11   출근           래미안파크스위트아파트 102동       서울 광진구 광나루로 545  127.090882   
 12   출근            DMC한강자이더헤리티지아파트    경기 고양시 덕양구 대

In [ ]:
commute_routes.to_csv('weekdays_route_maps/최종_출근.csv',index=False)
leave_routes.to_csv('weekdays_route_maps/최종_퇴근.csv',index=False)

In [ ]:
commute_routes

,type,출발_건물명,출발_주소,출발_lon,출발_lat,경유_도로명,경유_lon,경유_lat,도착_건물명,도착_주소,도착_lon,도착_lat
0,출근,당산래미안아파트 남문,,126.902219,37.531294,강변북로(난지한강공원),126.870423,37.573624,위프라임트윈타워,경기 고양시 덕양구 으뜸로 130,126.870517,37.579392
1,출근,문정래미안아파트 120동,서울 송파구 문정로 83,127.129843,37.489557,강변북로(난지한강공원),126.870423,37.573624,국제텔레시스 본사,경기 고양시 덕양구 으뜸로 124,126.869746,37.579115
2,출근,래미안파크스위트아파트 후문,서울 광진구 광나루로 545,127.089418,37.545448,강변북로(난지한강공원),126.870423,37.573624,천강이엔씨 본사,경기 고양시 덕양구 청초로 66,126.862946,37.578039
3,출근,동부센트레빌아파트 상가,서울 강서구 방화대로45길 32,126.814068,37.574401,강변북로(난지한강공원),126.870423,37.573624,라인업코리아 본사,경기 고양시 덕양구 청초로 66,126.861926,37.578035
4,출근,강서힐스테이트아파트 게이트1,,126.837658,37.547911,강변북로(난지한강공원),126.870423,37.573624,유어로스팅파크 본사,경기 고양시 덕양구 청초로 66,126.862883,37.577976
5,출근,남서울힐스테이트아파트,서울 금천구 시흥대로 165,126.901097,37.446174,올림픽대로,126.971274,37.508909,한일빌딩,서울 동작구 서달로15길 23,126.961735,37.508570
6,출근,래미안장안2차아파트 215동,서울 동대문구 장안벚꽃로 167,127.074949,37.572995,올림픽대로,126.971274,37.508909,덕봉빌딩,서울 동작구 서달로 166,126.962876,37.507753
7,출근,래미안빌라,서울 양천구 목동중앙로3길 83,126.867255,37.538523,올림픽대로,126.971274,37.508909,DR빌딩,서울 동작구 현충로 99,126.964882,37.508157
8,출근,잠실래미안아이파크아파트 123동,,127.108773,37.518259,올림픽대로,126.971274,37.508909,삼원빌딩,서울 동작구 서달로14가길 5,126.963106,37.507562
9,출근,평창동롯데캐슬로잔아파트,서울 종로구 평창문화로 156,126.977399,37.609980,올림픽대로,126.971274,37.508909,중앙타워빌,서울 동작구 흑석로 112,126.961627,37.508365


In [ ]:
leave_routes

,type,출발_건물명,출발_주소,출발_lon,출발_lat,경유_도로명,경유_lon,경유_lat,도착_건물명,도착_주소,도착_lon,도착_lat
0,퇴근,위프라임트윈타워,경기 고양시 덕양구 으뜸로 130,126.870517,37.579392,강변북로(난지한강공원),126.870423,37.573624,남서울힐스테이트아파트 상가,서울 금천구 시흥대로 165,126.901993,37.447228
1,퇴근,유어로스팅파크 본사,경기 고양시 덕양구 청초로 66,126.862883,37.577976,강변북로(난지한강공원),126.870423,37.573624,평창동롯데캐슬로잔아파트,서울 종로구 평창문화로 156,126.977399,37.609980
2,퇴근,라인업코리아 본사,경기 고양시 덕양구 청초로 66,126.861926,37.578035,강변북로(난지한강공원),126.870423,37.573624,목동삼성래미안2차아파트,서울 양천구 목동남로4길 81,126.867892,37.510062
3,퇴근,국제텔레시스 본사,경기 고양시 덕양구 으뜸로 124,126.869746,37.579115,강변북로(난지한강공원),126.870423,37.573624,래미안개포루체하임아파트,서울 강남구 개포로110길 50,127.080005,37.489245
4,퇴근,천강이엔씨 본사,경기 고양시 덕양구 청초로 66,126.862946,37.578039,강변북로(난지한강공원),126.870423,37.573624,DMC자이1단지아파트,서울 은평구 수색로 217,126.898650,37.579593
5,퇴근,HJ일화타워 주차장,서울 강동구 고덕비즈밸리로2가길 27,127.160230,37.566361,올림픽대로(강일IC),127.155176,37.572572,방화동부센트레빌아파트,서울 강서구 방화대로45길 32,126.814977,37.574702
6,퇴근,KS타워,서울 강동구 고덕비즈밸리로 38,127.159495,37.566337,올림픽대로(강일IC),127.155176,37.572572,철산래미안자이아파트 후문,경기 광명시 디지털로 56,126.873656,37.470574
7,퇴근,도드람타워 민영 주차장,서울 강동구 고덕비즈밸리로2가길 20,127.160784,37.566469,올림픽대로(강일IC),127.155176,37.572572,래미안파크스위트아파트,서울 광진구 광나루로 545,127.090375,37.546028
8,퇴근,도드람타워,서울 강동구 고덕비즈밸리로2가길 20,127.161007,37.566496,올림픽대로(강일IC),127.155176,37.572572,철산래미안자이아파트 게이트2,경기 광명시 디지털로 56,126.873571,37.470588
9,퇴근,HJ일화타워,서울 강동구 고덕비즈밸리로2가길 27,127.160039,37.566366,올림픽대로(강일IC),127.155176,37.572572,서초래미안아파트 남문,서울 서초구 서초대로65길 13-10,127.019473,37.497556


In [17]:
import folium

# ✅ 출근 / 퇴근 데이터 분리
# commute_df_only = commute_routes_all[commute_routes_all["type"]=="출근"].reset_index(drop=True)
# leave_df_only = routes_df[routes_df["type"]=="퇴근"].reset_index(drop=True)

# ✅ 출근 / 퇴근 데이터 분리
commute_df_only = commute_routes
leave_df_only = leave_routes

# ✅ 출근 지도
m_commute = folium.Map(location=[37.5665, 126.9780], zoom_start=11)

for idx, row in commute_df_only.iterrows():
    folium.PolyLine(
        locations=[[row["출발_lat"], row["출발_lon"]],
                   [row["경유_lat"], row["경유_lon"]],
                   [row["도착_lat"], row["도착_lon"]]],
        color='gray', weight=2.5, opacity=0.8
    ).add_to(m_commute)
    
    folium.Marker(
        location=[row["출발_lat"], row["출발_lon"]],
        popup=f"출근 출발: {row['출발_건물명']}",
        icon=folium.Icon(color='red', icon='play', prefix='fa')
    ).add_to(m_commute)
    
    folium.Marker(
        location=[row["경유_lat"], row["경유_lon"]],
        popup=f"경유: {row['경유_도로명']}",
        icon=folium.Icon(color='orange', icon='road', prefix='fa')
    ).add_to(m_commute)
    
    folium.Marker(
        location=[row["도착_lat"], row["도착_lon"]],
        popup=f"출근 도착: {row['도착_건물명']}",
        icon=folium.Icon(color='blue', icon='flag', prefix='fa')
    ).add_to(m_commute)

# ✅ 퇴근 지도
m_leave = folium.Map(location=[37.5665, 126.9780], zoom_start=11)

for idx, row in leave_df_only.iterrows():
    folium.PolyLine(
        locations=[[row["출발_lat"], row["출발_lon"]],
                   [row["경유_lat"], row["경유_lon"]],
                   [row["도착_lat"], row["도착_lon"]]],
        color='gray', weight=2.5, opacity=0.8
    ).add_to(m_leave)
    
    folium.Marker(
        location=[row["출발_lat"], row["출발_lon"]],
        popup=f"퇴근 출발: {row['출발_건물명']}",
        icon=folium.Icon(color='red', icon='play', prefix='fa')
    ).add_to(m_leave)
    
    folium.Marker(
        location=[row["경유_lat"], row["경유_lon"]],
        popup=f"경유: {row['경유_도로명']}",
        icon=folium.Icon(color='orange', icon='road', prefix='fa')
    ).add_to(m_leave)
    
    folium.Marker(
        location=[row["도착_lat"], row["도착_lon"]],
        popup=f"퇴근 도착: {row['도착_건물명']}",
        icon=folium.Icon(color='blue', icon='flag', prefix='fa')
    ).add_to(m_leave)



# 맨 위 코드 끝나고
m_commute.save("weekdays_route_maps/최종_출근_경로_지도.html")
m_leave.save("weekdays_route_maps/최종_퇴근_경로_지도.html")

# ✅ 지도 보기 (Jupyter Notebook 에서)
m_commute


In [16]:
m_leave